In [271]:
import numpy as np
import string
import math
import csv
from sklearn.model_selection import train_test_split
from sklearn import tree
from collections import Counter

In [272]:
class node:
    def __init__(self, nodeL=None, nodeR=None, threshold=None, column=None, value=None , majority=None):
        self.nodeL=nodeL
        self.nodeR=nodeR
        self.threshold=threshold
        self.column=column 
        self.value=value
        self.majority=majority
        

In [273]:
def split(X,y,mean,split_feature):
        XRight=np.empty([0,len(X[0])])
        YRight=np.array([])
        XLeft=np.empty([0,len(X[0])])
        YLeft=np.array([])
        for i in range(len(X)):
            if X[i][split_feature]<mean:
                XRight=np.append(XRight,[X[i]], axis=0)
                YRight=np.append(YRight,y[i]) 
            else:
                XLeft=np.append(XLeft,[X[i]], axis=0) 
                YLeft=np.append(YLeft,y[i]) 

        return XRight,YRight,XLeft,YLeft

In [274]:
def entropy(y):
    total = len(y)
    labels,count=np.unique(y, return_counts = True)
    probabil=[]
    
    for i in range(len(labels)):
        probabil.append(count[i]/total)
            
    
    entro=0
    for i in range(len(probabil)):
        entro+=probabil[i]*math.log2(probabil[i])
    entro=-1*entro
    return entro 

In [275]:
def gini(y):
    total = len(y)
    labels,count=np.unique(y, return_counts = True)
    probabil=[]
    
    for i in range(len(labels)):
        probabil.append(count[i]/total)
        
    gini_index=0
    for i in range(len(probabil)):
        gini_index+=(probabil[i])**2
    gini_index = 1- gini_index
    
    return gini_index 

In [276]:
def gain (X,Y,previous_entropy):
    gain_list=[]
    for i in range(len(X[0])) :
        mean = sum(X[:,i])/len(X)
        XRight,YRight,XLeft,YLeft=split(X,Y,mean,i)
        entropyright=entropy(YRight) 
        entropyleft=entropy(YLeft) 
        total=len(XRight)+len(XLeft)
        gain=previous_entropy-(((len(XRight)/total)*entropyright)+((len(XLeft)/total)*entropyleft))
        gain_list.append(gain)
    feature_selected=gain_list.index(max(gain_list))
    return feature_selected

In [277]:
def gini_information (X,Y): 
    gini_list=[]
    for i in range(len(X[0])) :
        mean = sum(X[:,i])/len(X)
        XRight,YRight,XLeft,YLeft=split(X,Y,mean,i)
        gini_right = gini (YRight)
        gini_left = gini (YLeft)
        gini_total = (len(XRight)*gini_right + len(XLeft)*gini_left)/len(Y)
        gini_list.append(gini_total)
    gini_feature = gini_list.index(min(gini_list))
    return gini_feature

In [278]:
def buildtree(X,y, impurity_measure):

    unique_X=np.unique(X, return_counts = True, axis = 0)
    
    if len(np.unique(y))==1:
        return node(value=y[0])
    
    elif len(unique_X) == 1 :
        #calculate most common label
        majority=Counter(y).most_common(1)[0][0]
        return node(value=majority)
    else:
        
        majority=Counter(y).most_common(1)[0][0]
        if impurity_measure =='entropy':
            #cal entropy
            previous_entropy = entropy(y)
            #cal gain
            split_feature=gain(X,y,previous_entropy)
        elif impurity_measure =='gini':
            split_feature=gini_information (X,y)
        

        #prepare to split 
        mean = sum(X[:,split_feature])/len(X)
        
        #split
        XRight,YRight,XLeft,YLeft=split(X,y,mean,split_feature)

        #Repeat
        if len(XRight)!=0:
                rightnode=buildtree(XRight,YRight,impurity_measure)

        if len(XLeft)!=0:
            leftnode=buildtree(XLeft,YLeft,impurity_measure)
    
        return node(nodeL=leftnode, nodeR=rightnode, threshold=mean, column=split_feature,majority=majority)

In [279]:
def learn(X, y,impurity_measure : string="entropy", pruning : bool=False):
    X_train, X_prune, Y_train, Y_prune = train_test_split(X,y,test_size = 0.3,
                                                            train_size = 0.7,random_state =125, shuffle = True)
    tree=buildtree(X_train,Y_train, impurity_measure)
    #if pruning true-> change tree to result of pruning
    if pruning :
        prune(tree,X_prune,Y_prune);
    
    return tree
    

In [280]:
def prune(tree,X,Y):
    c=Counter(Y)
    if tree.value != None :
        accuracy=c[tree.value]/len(Y)
        return accuracy 
    
    elif tree.value ==None:
        XRight,YRight,XLeft,YLeft=split(X,Y,tree.threshold,tree.column)
        if len(XRight)>0 :
            rightaccuracy=prune(tree.nodeR,XRight,YRight)
        else :
            rightaccuracy=0
            
        if len(XLeft)>0:
            leftaccuracy=prune(tree.nodeL,XLeft,YLeft)
        else :
            leftaccuracy=0
        childern_accuracy=((len(YRight)*rightaccuracy)+(len(YLeft)*leftaccuracy))/len(Y)
        node_accuracy=c[tree.majority]/len(Y)
        
        if node_accuracy>=childern_accuracy:
            tree.value=tree.majority
            tree.nodeR=None
            tree.nodeL=None
            return node_accuracy
        
        return childern_accuracy
        

In [281]:
def predict(X,tree):
    Y_predict=[]
    for row in range(len(X)):
        label=predicter(X[row],tree)
        Y_predict.append(label)
    
    return np.array(Y_predict)

In [282]:
def predicter(x,tree):
    if tree.value != None:
        return tree.value 
    elif x[tree.column]<tree.threshold:
        return predicter(x,tree.nodeR)
    else:
        return predicter(x,tree.nodeL)

In [283]:
def readfile():
    with open('magic04.data', 'r') as reading:
        innerlist = csv.reader(reading)
        data = list(innerlist)
    return np.array(data)

In [284]:
data=readfile()
X,Y=np.float_(data[:,:9]),data[:,10]

#split data
seed=129
X_train, X_val_test, Y_train, Y_val_test = train_test_split(X,Y,test_size = 0.3,
                                                            train_size = 0.7,random_state =seed, shuffle = True)

X_val, X_test, Y_val, Y_test = train_test_split(X_val_test,Y_val_test,test_size = 0.5, 
                                                              train_size = 0.5,random_state =seed, shuffle = True)

val_accuracy=[]

#entropy - no pruning 
tree1=learn(X_train,Y_train)

print("Entropy without pruning :")
Y_trained=predict(X_train,tree1)
correct_predictions = (Y_trained == Y_train)
training_accuracy = np.sum(correct_predictions) / len(X_train)
print("Training Accuracy : ", training_accuracy)

Y_predicted=predict(X_val,tree1)
good_predict_predictions = (Y_predicted == Y_val)
prediction_accuracy = np.sum(good_predict_predictions) / len(X_val)
val_accuracy.append(prediction_accuracy)
print("Validation Accuracy : ", prediction_accuracy ,'\n')

#entropy - pruning
tree2=learn(X_train,Y_train , pruning=True)
print("Entropy with pruning :")
Y_trained=predict(X_train,tree2)
correct_predictions = (Y_trained == Y_train)
training_accuracy = np.sum(correct_predictions) / len(X_train)
print("Training Accuracy : ", training_accuracy)

Y_predicted=predict(X_val,tree2)
good_predict_predictions = (Y_predicted == Y_val)
prediction_accuracy = np.sum(good_predict_predictions) / len(X_val)
val_accuracy.append(prediction_accuracy)
print("Validation Accuracy : ", prediction_accuracy ,'\n')

#gini - no pruning 
tree3=learn(X_train,Y_train , impurity_measure='gini')
print("Gini without pruning :")
Y_trained=predict(X_train,tree3)
correct_predictions = (Y_trained == Y_train)
training_accuracy = np.sum(correct_predictions) / len(X_train)
print("Training Accuracy : ", training_accuracy)

Y_predicted=predict(X_val,tree3)
good_predict_predictions = (Y_predicted == Y_val)
prediction_accuracy = np.sum(good_predict_predictions) / len(X_val)
val_accuracy.append(prediction_accuracy)
print("Validation Accuracy : ", prediction_accuracy ,'\n')

#gini - pruning
tree4=learn(X_train,Y_train,impurity_measure='gini',pruning=True)
print("Gini with pruning :")
Y_trained=predict(X_train,tree4)
correct_predictions = (Y_trained == Y_train)
training_accuracy = np.sum(correct_predictions) / len(X_train)
print("Training Accuracy : ", training_accuracy)

Y_predicted=predict(X_val,tree4)
good_predict_predictions = (Y_predicted == Y_val)
prediction_accuracy = np.sum(good_predict_predictions) / len(X_val)
val_accuracy.append(prediction_accuracy)
print("Validation Accuracy : ", prediction_accuracy ,'\n')


#selection 
best_model_index =val_accuracy.index(max(val_accuracy))

if(best_model_index == 0):
    print("The model selected is Entropy without pruning ")
    Y_tested=predict(X_test,tree1)
    good_test_predictions = (Y_tested == Y_test)
    test_accuracy = np.sum(good_test_predictions) / len(X_test)
    print("Test Accuracy : ", test_accuracy)
elif(best_model_index == 1):
    print("The model selected is Entropy with pruning ")
    Y_tested=predict(X_test,tree2)
    good_test_predictions = (Y_tested == Y_test)
    test_accuracy = np.sum(good_test_predictions) / len(X_test)
    print("Test Accuracy : ", test_accuracy)
elif(best_model_index == 2):
    print("The model selected is Gini without pruning ")
    Y_tested=predict(X_test,tree3)
    good_test_predictions = (Y_tested == Y_test)
    test_accuracy = np.sum(good_test_predictions) / len(X_test)
    print("Test Accuracy : ", test_accuracy)
elif(best_model_index == 3):
    print("The model selected is Gini with pruning ")
    Y_tested=predict(X_test,tree4)
    good_test_predictions = (Y_tested == Y_test)
    test_accuracy = np.sum(good_test_predictions) / len(X_test)
    print("Test Accuracy : ", test_accuracy)
    
#Comparison 
print("\n Sklearn Tree Classifier using Entropy : ")
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf = clf.fit(X_train, Y_train)
Y_predicted = clf.predict(X_val)
good_predict_predictions = (Y_predicted == Y_val)
prediction_accuracy = np.sum(good_predict_predictions) / len(X_val)
print("Validation Accuracy : ", prediction_accuracy)

Y_tested=clf.predict(X_test)
good_test_predictions = (Y_tested == Y_test)
test_accuracy = np.sum(good_test_predictions) / len(X_test)
print("Test Accuracy : ", test_accuracy)

print("\n Sklearn Tree Classifier using Gini : ")
clf = tree.DecisionTreeClassifier(criterion='gini')
clf = clf.fit(X_train, Y_train)
Y_predicted = clf.predict(X_val)
good_predict_predictions = (Y_predicted == Y_val)
prediction_accuracy = np.sum(good_predict_predictions) / len(X_val)
print("Validation Accuracy : ", prediction_accuracy)

Y_tested=clf.predict(X_test)
good_test_predictions = (Y_tested == Y_test)
test_accuracy = np.sum(good_test_predictions) / len(X_test)
print("Test Accuracy : ", test_accuracy)

Entropy without pruning :
Training Accuracy :  0.937884933153072
Validation Accuracy :  0.7991587802313355 

Entropy with pruning :
Training Accuracy :  0.8837314105452907
Validation Accuracy :  0.8412197686645636 

Gini without pruning :
Training Accuracy :  0.937434279705573
Validation Accuracy :  0.7974062390466176 

Gini with pruning :
Training Accuracy :  0.8837314105452907
Validation Accuracy :  0.8422712933753943 

The model selected is Gini with pruning 
Test Accuracy :  0.8293024886084823

 Sklearn Tree Classifier using Entropy : 
Validation Accuracy :  0.8107255520504731
Test Accuracy :  0.8194882579740624

 Sklearn Tree Classifier using Gini : 
Validation Accuracy :  0.8054679284963197
Test Accuracy :  0.8156326673676831
